In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import SnowballStemmer, LancasterStemmer

In [6]:
df = pd.read_csv("coffee_fix.csv")
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\r\n\r\n\r\n95\r\n\r\n\r\nA.R.C.\r\n“Sweety” ...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\r\n\r\n\r\n94\r\n\r\n\r\nA.R.C.\r\nFlora Ble...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRevel Coffee\r\nEth...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRoast House\r\nEthi...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\r\n\r\n\r\n94\r\n\r\n\r\nBig Creek Coffee Ro...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [7]:
tokens = [word_tokenize(i) for i in df["desc_1"]]
t1 = tokens[0]

In [8]:
t1

['Evaluated',
 'as',
 'espresso',
 '.',
 'Sweet-toned',
 ',',
 'deeply',
 'rich',
 ',',
 'chocolaty',
 '.',
 'Vanilla',
 'paste',
 ',',
 'dark',
 'chocolate',
 ',',
 'narcissus',
 ',',
 'pink',
 'grapefruit',
 'zest',
 ',',
 'black',
 'cherry',
 'in',
 'aroma',
 'and',
 'cup',
 '.',
 'Plush',
 ',',
 'syrupy',
 'mouthfeel',
 ';',
 'resonant',
 ',',
 'flavor-saturated',
 'finish',
 '.',
 'In',
 'three',
 'parts',
 'milk',
 ',',
 'rich',
 'chocolate',
 'tones',
 'intensify',
 ',',
 'along',
 'with',
 'intimations',
 'of',
 'vanilla',
 'paste',
 'and',
 'black',
 'cherry',
 'in',
 'the',
 'short',
 'finish',
 'and',
 'floral-toned',
 'citrus',
 'zest',
 'in',
 'the',
 'long',
 '.']

In [9]:
count = 0
for i in df["desc_1"]:
    if "three parts milk" in i:
        #print(i)
        count += 1
print(count)

210


In [10]:
def process_content(description):
    l = []
    try:
        words = nltk.word_tokenize(description.lower().replace("-"," "))
        #print(words)
        tagged = nltk.pos_tag(words)
        #print(tagged)

        # All chunks
        chunk_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*}"""
        chunk_parser = nltk.RegexpParser(chunk_gram)
        chunked = chunk_parser.parse(tagged)

        # Finding the finish description
        finish_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<VBN>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*<IN>+<DT>+<JJ.?>*<NN>*}"""
        finish_parser = nltk.RegexpParser(finish_gram)
        finish_chunks = finish_parser.parse(tagged)

        # Finding the milk description
        
        # print(description)

        for chunk in chunked:
            if hasattr(chunk,"label"):
                if chunk.label() == "Chunk":
                    l.append(chunk)
            # print(chunk)
        #print("---Finish---")
        if len(finish_chunks) == 0:
            for chunk in chunked:
                if hasattr(chunk,"label"):
                    if chunk.label() == "Chunk":
                        for j in chunk:
                            print(j[0])
        # for chunk in finish_chunks:
        #     if hasattr(chunk,"label"):
        #         if chunk.label() == "Chunk":
        #             print(chunk)
        #             # for j in chunk:
        #             #     print(j[0])
        #print("---Finish done---")

    except Exception as e:
        print(str(e))

    return l

chunks = process_content(df["desc_1"][0])
chunks

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', [('toned', 'VBD')]),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('cup', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ'), ('mouthfeel', 'NN')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('flavor', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD'), ('finish', 'JJ')]),
 Tree('Chunk', [('parts', 'NNS'), ('milk', 'NN')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN'), ('tones', 'NNS')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'

In [11]:
useless = ["milk","short","long","cup","flavor",
    "flavour","www","http","visit","information"
    "cafe","coffee","coffees","blend","shot"]

characteristic_names = ["finish","tone","tones","toned","mouthfeel","paste"
    "note","notes"]

def clean_chunk(chunk):
    grammar = [i[1] for i in chunk]
    words = [i[0] for i in chunk]
    for i in characteristic_names:
        for j in chunk:
            if j[0] == i:
                chunk.remove(j)
    for i in useless:
        if i in "".join(words):
            return None
    return chunk

In [12]:
clean = [clean_chunk(i) for i in chunks if clean_chunk(i) is not None]
clean

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', []),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('floral', 'JJ')]),
 Tree('Chunk', [('citrus', 'NN'), ('zest', 'NN')])]

In [13]:
stemmer = PorterStemmer()
stemmer = SnowballStemmer(language = "english")

In [14]:
def extract_characteristics(raw):
    characteristics = []
    for chunk in raw:
        grammar = [i[1] for i in chunk]
        words = [i[0] for i in chunk]
        clean = clean_chunk(chunk)
        if clean is None:
            continue

        if len(clean) == 0:
            continue
        
        if len(clean) == 1:
            if grammar[0] != "VBD":
                characteristics.append(stemmer.stem(words[0]))
            continue

        if all(i=="NN" for i in grammar):
            characteristics.append(stemmer.stem(words[0]))
            continue
        
        elif grammar[0].startswith('JJ') and grammar[1].startswith('NN'):
            characteristics.append(stemmer.stem(words[1]))
            continue

        elif grammar[0].startswith('RB') and grammar[1].startswith('JJ'):
            characteristics.append(stemmer.stem(words[1]))
            continue
    return np.array(characteristics)

extract_characteristics(chunks)

array(['espresso', 'sweet', 'rich', 'chocolati', 'vanilla', 'chocol',
       'narcissus', 'grapefruit', 'cherri', 'aroma', 'plush', 'syrupi',
       'reson', 'chocol', 'intim', 'vanilla', 'cherri', 'floral',
       'citrus'], dtype='<U10')

In [15]:
df["characteristics"] = df["desc_1"].apply(lambda x:extract_characteristics(process_content(x)))

In [16]:
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,characteristics
0,https://www.coffeereview.com/review/sweety-esp...,\r\n\r\n\r\n95\r\n\r\n\r\nA.R.C.\r\n“Sweety” ...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"[espresso, sweet, rich, chocolati, vanilla, ch..."
1,https://www.coffeereview.com/review/flora-blen...,\r\n\r\n\r\n94\r\n\r\n\r\nA.R.C.\r\nFlora Ble...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","[espresso, floral, honeysuckl, oak, chocol, th..."
2,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRevel Coffee\r\nEth...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","[crispli, cocoa, blossom, date, rice, pepperco..."
3,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRoast House\r\nEthi...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,...,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"[delic, peppercorn, date, myrrh, lavend, aroma..."
4,https://www.coffeereview.com/review/ethiopia-g...,\r\n\r\n\r\n94\r\n\r\n\r\nBig Creek Coffee Ro...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,...,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"[sweet, pungent, honey, pear, tangerin, zest, ..."


In [17]:
all_characteristics = set([])
for i in df["characteristics"]:
    all_characteristics.update(i)

all_characteristics

{'access',
 'acid',
 'acrid',
 'addit',
 'adult',
 'agav',
 'akin',
 'alcohol',
 'aliv',
 'all',
 'allspic',
 'almond',
 'almondi',
 'almost',
 'aloewood',
 'also',
 'alyssum',
 'amber',
 'amond',
 'amplif',
 'amplifi',
 'anchor',
 'anis',
 'appeal',
 'appear',
 'appl',
 'apricot',
 'aprium',
 'aroma',
 'aromat',
 'array',
 'assess',
 'attract',
 'author',
 'autumn',
 'azalea',
 'back',
 'backdrop',
 'background',
 'bacon',
 'bake',
 'baker',
 'balanc',
 'ballast',
 'balm',
 'balsa',
 'banana',
 'barley',
 'barrel',
 'base',
 'bay',
 'bean',
 'beauti',
 'beef',
 'bergamot',
 'berri',
 'beverag',
 'big',
 'bight',
 'bing',
 'bit',
 'bitter',
 'bitter/tart',
 'bitterish',
 'bittersweet',
 'bizarr',
 'blackberri',
 'blackcurr',
 'blood',
 'bloom',
 'blossom',
 'blueberri',
 'bodi',
 'bold',
 'bosc',
 'bottom',
 'bouquet',
 'bourbon',
 'boysenberri',
 'brace',
 'brandi',
 'brazil',
 'breadfruit',
 'brew',
 'bright',
 'brilliant',
 'brisk',
 'brittl',
 'brothi',
 'brown',
 'bubblegum',
 'bu

In [18]:
df["coffee"] = df["name"] + "-" + df["roaster"]
df.head(3)

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,characteristics,coffee
0,https://www.coffeereview.com/review/sweety-esp...,\r\n\r\n\r\n95\r\n\r\n\r\nA.R.C.\r\n“Sweety” ...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"[espresso, sweet, rich, chocolati, vanilla, ch...",“Sweety” Espresso Blend-A.R.C.
1,https://www.coffeereview.com/review/flora-blen...,\r\n\r\n\r\n94\r\n\r\n\r\nA.R.C.\r\nFlora Ble...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","[espresso, floral, honeysuckl, oak, chocol, th...",Flora Blend Espresso-A.R.C.
2,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRevel Coffee\r\nEth...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","[crispli, cocoa, blossom, date, rice, pepperco...",Ethiopia Shakiso Mormora-Revel Coffee


In [120]:
# We want to remove characteristics that appear in less that 10% of the coffees
d = {}
for i in df["characteristics"]:
    for j in i:
        if j in d:
            d[j] += 1
        else:
            d[j] = 1

count_dict = {i:j for i,j in d.items() if j > len(df)/10}
count_dict

{'espresso': 315,
 'sweet': 1195,
 'rich': 415,
 'chocolati': 398,
 'chocol': 1558,
 'narcissus': 256,
 'cherri': 238,
 'aroma': 2255,
 'plush': 309,
 'syrupi': 536,
 'reson': 279,
 'floral': 487,
 'oak': 235,
 'gentl': 373,
 'crispli': 244,
 'cocoa': 1070,
 'delic': 230,
 'finish': 511,
 'hint': 433,
 'crisp': 536,
 'acid': 1239,
 'zest': 334,
 'juici': 553,
 'structur': 691,
 'almond': 579,
 'sandalwood': 340,
 'fruit': 435,
 'high': 271,
 'apricot': 267,
 'magnolia': 291,
 'savori': 265,
 'full': 514,
 'cedar': 761,
 'cut': 317,
 'tart': 468,
 'hazelnut': 358,
 'satini': 354,
 'flower': 351,
 'underton': 256,
 'velveti': 293,
 'bright': 376}

In [121]:
one_hot = pd.DataFrame(columns = ["coffee"] + list(count_dict.keys()))
one_hot

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright


In [122]:
d = {"coffee":df["coffee"]}

for i in count_dict.keys():
    d[i] = df["characteristics"].apply(lambda x: 0 if i in x else 1)

In [123]:
one_hot = pd.DataFrame(d)
one_hot.head()

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright
0,“Sweety” Espresso Blend-A.R.C.,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,Flora Blend Espresso-A.R.C.,0,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2,Ethiopia Shakiso Mormora-Revel Coffee,1,1,0,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,Ethiopia Suke Quto-Roast House,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,Ethiopia Gedeb Halo Beriti-Big Creek Coffee Ro...,1,0,1,1,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1


In [124]:
from sklearn.neighbors import NearestNeighbors

In [125]:
one_hot[one_hot.columns[1:]]

,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,syrupi,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,0,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
2,1,1,0,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,0,1,1,0,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,1,1,1,1,0,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
2278,1,1,1,0,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
2279,1,0,1,1,1,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2280,1,0,1,0,0,1,1,0,1,0,...,1,1,1,0,0,1,1,1,1,1


In [132]:
X = one_hot[one_hot.columns[1:]]
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [133]:
import prince

In [141]:
# We want to reduce the number of features but limit the difference in nearest neighbours
# We stop once only 10% of the sets have no overlap
l = [0]
n = 4
while True:
    n+=1
    mca = prince.MCA(n_components=n)
    mca = mca.fit(X)
    mca = mca.transform(X)
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(mca)
    mca_distances, mca_indices = nbrs.kneighbors(mca)
    l = []
    for i in range(len(indices)):
        l.append(len(set(indices[i][1:]) & set(mca_indices[i][1:])))
    if l.count(0) < len(l)/10:
        break
print(n)

20


In [142]:
mca = prince.MCA(n_components=n)
mca = mca.fit(X)
mca = mca.transform(X)
mca

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.025074,0.318228,0.047341,0.123570,0.224134,-0.073037,-0.137454,0.100727,-0.154271,0.011991,0.082136,-0.159211,0.054945,0.100310,0.130425,0.074101,-0.055986,0.024102,0.005200,0.022703
1,-0.021282,0.241927,-0.002136,-0.087406,0.057331,0.019256,-0.087874,0.129129,-0.052719,0.005745,0.049171,-0.081433,0.054730,0.069654,-0.029038,0.076562,-0.049493,-0.045121,-0.062245,-0.007541
2,-0.023370,-0.031865,0.161014,-0.097921,-0.095761,0.092606,-0.029032,0.116824,0.046809,-0.205090,0.035968,-0.067468,-0.058238,-0.050597,0.068087,-0.110175,-0.086589,0.063870,0.039175,-0.065339
3,-0.026508,-0.088908,-0.047582,-0.105501,-0.107253,-0.014130,-0.076721,-0.052737,-0.013822,0.045566,0.082174,0.056708,-0.056477,-0.011290,0.023515,0.011758,0.008776,-0.022221,0.008635,0.002768
4,-0.012232,0.137956,0.082171,0.214607,0.128401,0.168808,-0.021615,-0.127138,-0.045665,-0.059751,-0.028723,-0.000017,-0.064667,-0.023304,-0.032902,-0.096107,0.013102,-0.096146,-0.120858,-0.038081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,-0.035554,-0.028238,-0.215958,-0.107268,0.085084,0.165797,0.070560,0.010054,-0.008373,-0.034443,0.158409,-0.009939,0.138157,-0.063073,-0.109730,-0.015707,0.022129,0.058207,-0.060727,-0.047295
2278,-0.029716,-0.125359,-0.049122,-0.087392,-0.142736,0.106244,0.070981,0.044178,-0.137390,0.119198,-0.104366,-0.190335,0.044213,0.002566,-0.090297,-0.094640,-0.101829,0.004880,-0.033540,-0.088576
2279,-0.032637,-0.114409,-0.031698,0.096874,-0.044234,-0.132987,-0.083685,-0.056407,0.000113,0.088082,0.084779,0.046683,-0.049787,0.029113,0.004943,-0.001679,-0.005865,-0.054757,0.017214,0.058548
2280,-0.029880,0.020770,-0.158768,0.072235,0.176584,-0.140756,-0.062828,-0.058447,-0.044810,-0.034328,0.003077,0.063465,-0.018743,-0.059411,-0.000461,0.106972,-0.107260,0.071141,-0.080036,-0.046612


In [143]:
for i in mca_indices:
    print(i)

[   0  635  657  809 2189 1127]
[   1 1354  266 1740  809 2037]
[   2  992 1148 1031 1576 1502]
[   3 2245 2138 1821 1764 2002]
[  4  31  98  10 172 293]
[   5 1159 1398 1490  361 2208]
[   6  513  507  514 1195  815]
[   7 1145  296 1717 1576 1768]
[   8 1719   36 1443  576  969]
[   9 1099  145 2002  472 1131]
[ 10  65  57 262 310 254]
[  11   56  549  253 1874 1232]
[  12 1710  171  205  577  578]
[  13  296    7  129   61 1034]
[ 14 243 708 433 790 691]
[  15 1658  789  773  837 1866]
[  16 2253 1748 1814  339 2034]
[  17 1087  992 1940 1717 1576]
[  18  165  296 1183  443  663]
[  19  837 2082  789  281  170]
[  20  574  563   48   33 1265]
[  21 1547 2031 1593 1560 1130]
[  22 1265  589  793  316 1382]
[  23  197  691  701 1991  774]
[  24 2018  750 2144  259 1485]
[  25  315 1952  168  253  310]
[  26 1465  395  363 1015 2095]
[ 27 517  42 346 315 666]
[  28 2248 2256 2268 2002 1255]
[  29 1290   75 1572 1872 2124]
[  30 1528 1031  395 1749  562]
[ 31   4 172 293 191  35]
[  32 